# Análisis de nuevo dataset

In [1]:
import pymongo
from bson.objectid import ObjectId
import os
import random
from twython import Twython
import pandas as pd
import time

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Span
import matplotlib.pyplot as plt

output_notebook()

Loading BokehJS ...

In [2]:

#conexion mongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["twitter-memoria"]
coleccion_completa = mydb["nuevos_tweets"]
bd_hilos = mydb["nuevos_hilos"]
usuarios = mydb["nuevos_usuarios"]

#obtener llaves Twitter
llaves_bd = mydb["llaves"]
llaves = llaves_bd.find_one()

APP_KEY = llaves["API_KEY"]
ACCESS_TOKEN  = llaves["ACCESS_TOKEN_T"]
twitter       = Twython(APP_KEY, access_token=ACCESS_TOKEN)

#datos
tweets = pd.DataFrame(list(coleccion_completa.find({})))
hilos = pd.DataFrame(list(bd_hilos.find({})))

tweets = tweets.drop(columns = ["user", "text", "in_response_to", "id_str", "hashtags", "created_at", "_id"]).sort_values("progreso_hilo")
hilos = hilos.drop(columns = ["_id", "texto", "usuario"])


In [3]:
hilos.head()

,all_hashtags,emocion_max,emocion_media,emocion_moda,emocion_presente,hilo,promedio_favorites,promedio_retweet,puntaje_sigmoideo,total_favorites,total_hashtags,total_retweet,total_tweets
0,[],"[Fear, Joy]","[Fear, Joy]","[Joy, Disgust]","[Fear, Joy]",0,16,10,1.26775,49,0,30,3
1,[],"[Anger, Surprise]","[Joy, Anger]","[Joy, Anger]","[Joy, Anger]",1,31488,10485,1.16106,94463,0,31455,3
2,[],"[Sadness, Surprise]","[Joy, Fear]","[Joy, Sadness]","[Joy, Fear]",2,183,55,0.97446,549,0,165,3
3,[],"[Joy, Surprise]","[Joy, Surprise]","[Joy, Surprise]","[Surprise, Joy]",3,5014,2091,1.17440,15043,0,6274,3
4,"[DemDebate2, BLEXIT]","[Joy, Fear]","[Joy, Fear]","[Joy, Fear]","[Joy, Fear]",4,13153,5852,1.22231,39459,2,17557,3


In [4]:
tweets.head()

,aporte_favoritos,aporte_retweet,favorite_count,hashtag_count,hilo,id,pos_hilo,progreso_hilo,puntaje_sigmoideo,retweet_count
6131,0.102,0.144,164,0,392.0,1142603256616480768,0.0,1.0,0.97981,126
5146,0.119,0.198,1387,1,276.0,1106705920753463297,0.0,1.0,1.17762,1315
1936,0.760,0.745,6562,0,33.0,1124435158101315593,0.0,1.0,1.04774,4870
1938,0.438,0.522,64,0,34.0,1133680310313529344,0.0,1.0,0.87802,36
4425,0.270,0.364,355,0,213.0,1069455259922120704,0.0,1.0,0.84641,172


In [5]:
progreso = tweets["progreso_hilo"].tolist()
aporte_fv = tweets["aporte_favoritos"].tolist()
aporte_rt = tweets["aporte_retweet"].tolist()

In [8]:

grafico1 = figure(plot_width=700, 
           plot_height=350, 
           title='Probabilidad de cada emoción por tweet', 
           background_fill_color="#f2f3f7", 
           y_axis_label='probabilidad', 
           x_axis_label='Tweets along the thread')

grafico2 = figure(plot_width=700, 
           plot_height=350, 
           title='Probabilidad de cada emoción por tweet', 
           background_fill_color="#f2f3f7", 
           y_axis_label='probabilidad', 
           x_axis_label='Tweets along the thread')

grafico1.circle(progreso, aporte_fv, line_color='#cd2626', legend='aporte_fv')
grafico2.circle(progreso, aporte_rt, line_color='#64bd40', legend='aporte_rt')

show(grafico1)
show(grafico2)

In [9]:
grup_tweet = tweets.groupby(["progreso_hilo"]).mean()

In [13]:
grup_progreso = grup_tweet.index.tolist()
grup_aporte_fv = grup_tweet["aporte_favoritos"].tolist()
grup_aporte_rt = grup_tweet["aporte_retweet"].tolist()

In [17]:
grafico1 = figure(plot_width=700, 
           plot_height=350, 
           title='Probabilidad de cada emoción por tweet', 
           background_fill_color="#f2f3f7", 
           y_axis_label='probabilidad', 
           x_axis_label='Tweets along the thread')

grafico2 = figure(plot_width=700, 
           plot_height=350, 
           title='Probabilidad de cada emoción por tweet', 
           background_fill_color="#f2f3f7", 
           y_axis_label='probabilidad', 
           x_axis_label='Tweets along the thread')

grafico1.line(grup_progreso, grup_aporte_fv, line_color='#cd2626', legend='aporte_fv')
grafico1.line(grup_progreso, grup_aporte_rt, line_color='#64bd40', legend='aporte_rt')

show(grafico1)